# Iteration 2: Introduction to LangGraph

In this iteration, we'll convert our baseline BS detector to use LangGraph, adding retry logic and learning the core concepts.

## Learning Objectives
- Understand LangGraph's 5 core concepts: State, Node, Edge, Routing, Graph
- Convert existing code to graph-based architecture
- Add retry logic using graph cycles
- Learn two execution patterns: single run and chat loop

## Architecture Overview

Let's visualize what we're building:

In [49]:
%%html
<script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
<script>
mermaid.initialize({ 
    startOnLoad: true,
    theme: 'default',
    themeVariables: {
        fontSize: '16px'
    }
});
</script>

In [50]:
import base64
from IPython.display import Image, display

# Define the mermaid diagram
mermaid_graph = """
graph TD
    Start([Start]) --> Detect[Detect BS Node]
    Detect --> Router{Route Decision}
    Router -->|Success| Format[Format Output]
    Router -->|Retry| Retry[Retry Node]
    Router -->|Max Retries| Format
    Retry --> Detect
    Format --> End([End])

    style Start fill:#90EE90
    style End fill:#FFB6C1
    style Detect fill:#87CEEB
    style Retry fill:#FFE4B5
"""

# Use mermaid.ink API to render the diagram
def render_mermaid_diagram(graph_def):
    """Render mermaid diagram using mermaid.ink API"""
    graph_bytes = graph_def.encode("utf-8")
    base64_string = base64.b64encode(graph_bytes).decode("ascii")
    image_url = f"https://mermaid.ink/img/{base64_string}?type=png"
    
    # Display the image
    return Image(url=image_url)

# Display the diagram
display(render_mermaid_diagram(mermaid_graph))

## 1. Setup and Imports

In [51]:
import sys
sys.path.append('..')

# Import our baseline detector from Iteration 1
from modules.m1_baseline import BSDetectorOutput, check_claim
from modules.m2_langgraph import (
    BSDetectorState,
    create_bs_detector_graph,
    check_claim_with_graph,
    visualize_graph
)
from config.llm_factory import LLMFactory

# Create LLM instance
llm = LLMFactory.create_llm()
print(f"Using LLM: {llm.__class__.__name__}")

Using LLM: ChatOpenAI


## 2. Understanding LangGraph Concepts

### Concept 1: State
State is the shared data between nodes. We use Pydantic BaseModel for validation and type safety!

In [52]:
from typing import Optional
from pydantic import BaseModel, Field

# Let's look at our state definition using Pydantic
class BSDetectorState(BaseModel):
    """Everything our graph needs to remember - with validation!"""
    # Input
    claim: str
    
    # Processing control
    retry_count: int = 0
    max_retries: int = 3
    
    # Output
    verdict: Optional[str] = None
    confidence: Optional[int] = None
    reasoning: Optional[str] = None
    error: Optional[str] = None
    result: Optional[dict] = None

# Example state - now with validation!
example_state = BSDetectorState(
    claim="The Boeing 747 can fly backwards",
    retry_count=0,
    max_retries=3
)
print("Example state:", example_state.model_dump())
print("State type:", type(example_state))

Example state: {'claim': 'The Boeing 747 can fly backwards', 'retry_count': 0, 'max_retries': 3, 'verdict': None, 'confidence': None, 'reasoning': None, 'error': None, 'result': None}
State type: <class '__main__.BSDetectorState'>


### Concept 2: Node
A node is just a function that takes state and returns updates!

In [53]:
def simple_node(state: BSDetectorState) -> dict:
    """A node processes state and returns updates"""
    claim = state.claim  # Access fields as attributes
    
    # Do some processing
    if "backwards" in claim.lower():
        verdict = "BS"
    else:
        verdict = "LEGITIMATE"
    
    # Return ONLY the fields to update
    return {
        "verdict": verdict,
        "confidence": 95
    }

# Test the node
updates = simple_node(example_state)
print("Node returns updates:", updates)
print("Original state is unchanged:", example_state.verdict)

Node returns updates: {'verdict': 'BS', 'confidence': 95}
Original state is unchanged: None


### Concept 3: Routing
Routing functions decide which path to take

In [54]:
def routing_function(state: BSDetectorState) -> str:
    """Decide next step based on state"""
    if state.verdict:
        return "success"
    elif state.retry_count < state.max_retries:
        return "retry"
    else:
        return "give_up"

# Test routing with different states
test_states = [
    BSDetectorState(claim="test", verdict="BS", retry_count=0, max_retries=3),
    BSDetectorState(claim="test", verdict=None, retry_count=1, max_retries=3),
    BSDetectorState(claim="test", verdict=None, retry_count=3, max_retries=3)
]

for state in test_states:
    route = routing_function(state)
    print(f"Verdict: {state.verdict}, Retries: {state.retry_count}/{state.max_retries} → Route: {route}")

Verdict: BS, Retries: 0/3 → Route: success
Verdict: None, Retries: 1/3 → Route: retry
Verdict: None, Retries: 3/3 → Route: give_up


## 3. Building a Simple Graph Step-by-Step

In [55]:
from langgraph.graph import StateGraph, END

# Step 1: Create a graph with our Pydantic state
graph = StateGraph(BSDetectorState)

# Step 2: Add nodes (we'll use simplified versions)
def detect_node(state: BSDetectorState) -> dict:
    print(f"🔍 Detecting BS for: {state.claim[:30]}...")
    # Simulate detection
    return {"verdict": "BS", "confidence": 90}

def retry_node(state: BSDetectorState) -> dict:
    print(f"🔄 Retry {state.retry_count + 1}")
    return {"retry_count": state.retry_count + 1}

graph.add_node("detect", detect_node)
graph.add_node("retry", retry_node)

# Step 3: Set entry point
graph.set_entry_point("detect")

# Step 4: Add routing
def simple_router(state: BSDetectorState) -> str:
    if state.verdict:
        return "done"
    return "retry"

graph.add_conditional_edges(
    "detect",
    simple_router,
    {"done": END, "retry": "retry"}
)

# Step 5: Connect retry back to detect
graph.add_edge("retry", "detect")

# Step 6: Compile
simple_app = graph.compile()

print("✅ Graph compiled successfully!")

# Step 7: Visualize the graph we just built
print("\n📊 Here's what we built:")
mermaid_code = simple_app.get_graph().draw_mermaid()
display(render_mermaid_diagram(mermaid_code))

✅ Graph compiled successfully!

📊 Here's what we built:


## 4. Compare Baseline vs LangGraph

### 🔹 Baseline Version (no retry)
First, let's see how our original baseline detector handles the claim:

### 🔹 LangGraph Version (with retry)
Now let's see how the LangGraph version handles the same claim with retry logic:

In [ ]:
test_claim = "The Airbus A380 can do barrel rolls"

# Baseline Version (no retry)
baseline_result = check_claim(test_claim, llm)
print(f"Verdict: {baseline_result['verdict']}")
print(f"Confidence: {baseline_result['confidence']}%")
print(f"Reasoning: {baseline_result['reasoning']}")

print()  # Empty line for spacing

# LangGraph Version (with retry)
graph_result = check_claim_with_graph(test_claim, max_retries=3)
print(f"Verdict: {graph_result['verdict']}")
print(f"Confidence: {graph_result['confidence']}%")
print(f"Reasoning: {graph_result['reasoning']}")

## 5. Execution Pattern 1: Single Run

In [57]:
def single_run_demo():
    """Execute the graph once"""
    # Create the graph
    app = create_bs_detector_graph()
    
    # Test claims
    claims = [
        "The Boeing 747 has four engines",
        "Helicopters fly using jet propulsion",
        "The Concorde could break the sound barrier"
    ]
    
    for claim in claims:
        print(f"\nClaim: {claim}")
        
        # Initialize state with Pydantic model
        state = BSDetectorState(
            claim=claim,
            retry_count=0,
            max_retries=2
        )
        
        # Run the graph
        result = app.invoke(state)
        
        # Extract verdict from the result dictionary
        if result.get("result"):
            print(f"→ {result['result']['verdict']}")

single_run_demo()


Claim: The Boeing 747 has four engines
→ LEGITIMATE

Claim: Helicopters fly using jet propulsion
→ BS

Claim: The Concorde could break the sound barrier
→ LEGITIMATE


## 6. Execution Pattern 2: Interactive Chat

Now let's create a real interactive chat where you can keep entering claims until you're ready to exit. This shows how the graph can be reused for multiple queries in a session.

### 🤖 BS Detector Chat - Interactive Mode

Type aviation claims to check them. You can:
- Enter any aviation-related claim to verify
- Type 'quit', 'exit', or 'q' to stop the chat

**Note**: When you run the cell below, it will wait for your input. Type an aviation claim and press Enter. Keep entering claims until you type 'quit' to exit.

In [61]:
# Interactive Chat Interface
print("🤖 BS Detector Chat - Interactive Mode")
print("=" * 40)
print("Type aviation claims to check them.")
print("Type 'quit', 'exit', or 'q' to stop.")
print("=" * 40)

# Create the graph once
app = create_bs_detector_graph()

# Interactive loop
while True:
    # Get user input
    claim = input("\n💬 Enter claim: ").strip()
    
    # Check for exit
    if claim.lower() in ['quit', 'exit', 'q']:
        print("\n👋 Goodbye!")
        break
    
    # Skip empty input
    if not claim:
        continue
    
    print("\n🔍 Analyzing...")
    
    # Process with graph
    state = app.invoke(BSDetectorState(
        claim=claim,
        retry_count=0,
        max_retries=3
    ))
    
    # Show result
    result = state.get("result") or {}
    
    if result.get("verdict") and result["verdict"] != "ERROR":
        print(f"\n📊 Verdict: {result['verdict']}")
        print(f"🎯 Confidence: {result['confidence']}%")
        print(f"💭 Reasoning: {result['reasoning']}")
    else:
        print(f"\n❌ Error: {result.get('error', 'Unknown error')}")
    
    print("-" * 40)

🤖 BS Detector Chat - Interactive Mode
Type aviation claims to check them.
Type 'quit', 'exit', or 'q' to stop.



💬 Enter claim:  q



👋 Goodbye!


## 7. Visualizing the Graph

In [62]:
# Show the actual graph structure
print("Here's the complete BS Detector graph with all nodes:")
graph_image = visualize_graph()
if graph_image:
    display(graph_image)
else:
    print("(Visual display not available in this environment)")

Here's the complete BS Detector graph with all nodes:
BS Detector Graph Structure:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	detect(detect)
	retry(retry)
	format_output(format_output)
	__end__([<p>__end__</p>]):::last
	__start__ --> detect;
	detect -. &nbsp;error&nbsp; .-> format_output;
	detect -.-> retry;
	retry --> detect;
	format_output --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc


Graph Flow:
1. Start → Detect BS
2. If success → Format Output → End
3. If error & retries left → Retry → Detect BS
4. If max retries → Format Output → End
(Visual display not available in this environment)


## 8. Exercise: Add a Logging Node

Let's practice by adding a new node to our graph!

In [63]:
from langgraph.graph import StateGraph, END
import json

# Create a new graph with logging
def create_graph_with_logging():
    graph = StateGraph(BSDetectorState)
    
    # Reuse our existing nodes
    from modules.m2_langgraph import (
        detect_bs_node, 
        retry_node, 
        format_output_node,
        route_after_detection
    )
    
    # ADD YOUR LOGGING NODE HERE
    def log_node(state: BSDetectorState) -> dict:
        """Log the state for debugging"""
        print("\n📊 State Log:")
        print(f"  Claim: {state.claim[:50]}...")
        print(f"  Verdict: {state.verdict or 'None'}")
        print(f"  Retries: {state.retry_count}")
        return {}  # No state updates
    
    # Build the graph
    graph.add_node("detect", detect_bs_node)
    graph.add_node("retry", retry_node)
    graph.add_node("log", log_node)  # New!
    graph.add_node("format", format_output_node)
    
    # Entry point
    graph.set_entry_point("detect")
    
    # Routing
    graph.add_conditional_edges(
        "detect",
        route_after_detection,
        {
            "success": "log",  # Go to log first
            "retry": "retry",
            "error": "log"
        }
    )
    
    # Connect the rest
    graph.add_edge("retry", "detect")
    graph.add_edge("log", "format")  # Log then format
    graph.add_edge("format", END)
    
    return graph.compile()

# Test the enhanced graph
enhanced_app = create_graph_with_logging()
result = enhanced_app.invoke(BSDetectorState(
    claim="Airplanes can teleport between airports",
    retry_count=0,
    max_retries=1
))

print("\n✅ Graph with logging executed successfully!")

# Show the final result
if result.get("result"):
    print(f"Final verdict: {result['result']['verdict']}")


📊 State Log:
  Claim: Airplanes can teleport between airports...
  Verdict: BS
  Retries: 0

✅ Graph with logging executed successfully!
Final verdict: BS


## 9. Key Takeaways

### What We Learned:

1. **State** = Shared memory (Pydantic BaseModel for validation)
2. **Node** = Function that processes state
3. **Edge** = Connection between nodes
4. **Routing** = Conditional flow control
5. **Graph** = Complete system

### Benefits of LangGraph:
- ✅ Separation of concerns (each node has one job)
- ✅ Easy to add features (just add nodes)
- ✅ Built-in retry logic
- ✅ Visual representation of flow
- ✅ Testable components
- ✅ Type safety with Pydantic models

### Next Steps:
- Try adding more nodes (validation, caching, etc.)
- Experiment with parallel processing
- Build more complex routing logic

## 10. Challenge: Modify the Graph

Can you add a confidence adjustment node that:
- Reduces confidence by 10% for each retry
- Runs after detection but before formatting?

In [64]:
# Your solution here!
def confidence_adjustment_node(state: BSDetectorState) -> dict:
    """Adjust confidence based on retries"""
    if state.confidence and state.retry_count > 0:
        # Reduce confidence by 10% for each retry
        adjusted_confidence = max(0, state.confidence - (10 * state.retry_count))
        return {"confidence": adjusted_confidence}
    return {}  # No adjustment needed